___

<a href='https://www.udemy.com/user/joseportilla/'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Content Copyright by Pierian Data</em></center>

# Python Email Exercise Ideas

Since we can't really assess any code that would involve your personal email address, here are some ideas for you to test your new skills. Please keep in mind, we can not assess these.


## Ideas

* Daily Automatic Email Reminder for your Tasks
* Webscrape some statistics from a website automatically each day and email them to yourself
* Automatically email daily/weekly/monthly reports at your work
* Have end of day messages to your friends and family be sent out at random to spread joy
* Be creative! Mix together any of the skills you've learned so far with email :)

# Podcast Newsletter App

In [ ]:
'''
IDEA:

- create a Podcast newsletter for headlines from geopolitics, finance, science, tech, and AI.
    - websites:
        - geopolitics:
            - https://www.reuters.com/world/
            - https://ground.news/
        - finance
            - https://www.reuters.com/business/
            - https://www.reuters.com/markets/ 
        - science:
            - https://ground.news/interest/science
            - https://www.sciencenews.org/
        - tech: 
            - https://www.wired.com/
            - https://ground.news/interest/tech
        - AI:
            - https://ground.news/interest/ai
            - https://www.artificialintelligence-news.com/

- newsletter that will be sent every second day at 9am (server needed?)
- set up server on Google Cloud Run

'''

In [ ]:
'''
STEPS:

1. set up web scraper -> headlines, text
2. set up LLM (perplexity) integration -> short explainers for biggest headlines (let it choose which is biggest headline)
3. finalize text and add image as part of email
4. set up emailing infrastructure
5. integrate structure into Cloud Run

'''

In [ ]:
# TODO add error handling

import os
import requests
import bs4
import smtplib
import getpass
from PIL import Image

In [ ]:
# - geopolitics:
    # - https://www.reuters.com/world/
    # - https://ground.news/
# - finance
    # - https://www.reuters.com/business/
    # - https://www.reuters.com/markets/ 
# - science:
    # - https://ground.news/interest/science
    # - https://www.sciencenews.org/
# - tech: 
    # - https://www.wired.com/
    # - https://ground.news/interest/tech
# - AI:
    # - https://ground.news/interest/ai
    # - https://www.artificialintelligence-news.com/

In [ ]:
### 1 ### scrape headlines and first few lines

# FIX needed?

headlines_and_text = {} # dict for headlines and text


In [ ]:
# request websites

# reuters_geo = requests.get('https://www.reuters.com/world/')
ground_geo = requests.get('https://ground.news/')
# reuters_bus = requests.get('https://www.reuters.com/business/')
# reuters_fin = requests.get('https://www.reuters.com/markets/')
news_sci = requests.get('https://www.sciencenews.org/')
ground_sci = requests.get('https://ground.news/interest/science')
wired_tech = requests.get('https://www.wired.com/')
ground_tech = requests.get('https://ground.news/interest/tech')
ground_ai = requests.get('https://ground.news/interest/ai')
news_ai = requests.get('https://www.artificialintelligence-news.com/')

In [ ]:
# ground news

soup_ground_geo = bs4.BeautifulSoup(ground_geo.text, 'lxml')
soup_ground_geo

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link data-precedence="next" href="/_next/static/css/3da850aeb14025ee.css" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/d5430b1b737a17c6.css" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/64d5928f8e4d43c4.css" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/b7098ecb51a72646.css" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/f18e8dcd7e0f4a59.css" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/38c5bc079c41ea7d.css" rel="stylesheet"/><link as="script" fetchpriority="low" href="/_next/static/chunks/webpack-1523163af9a027df.js" rel="preload"/><script async="" src="/_next/static/chunks/4bd1b696-16dcc2bec6e1bff8.js"></script><script async="" src="/_next/static/chunks/1255-4f557b50c880e314.js"></script><script async="" src="/_next/static/chunks/

In [35]:
headlines_geo = soup_ground_geo.find_all('h4')
headlines_geo = [headline.getText() for headline in headlines_geo]
headlines_geo = headlines_geo[:4]

["First 'one in one out' migrant sent back to France, government says",
 'Lithuania charges 15 over alleged Russian-backed parcel bombs',
 'Madeleine McCann suspect Christian B released after serving rape sentence',
 'Charlie Kirk shooting suspect charged with aggravated murder; prosecutor to seek death penalty',
 'UN commission says Israel is committing genocide in Gaza',
 'Warsaw turns to Ukraine for drone warfare expertise after Russian drones enter Polish airspace',
 'Italy Passes National AI Law Aligning With EU Rules After Final Senate Vote',
 'Roche to acquire liver drug developer 89bio for up to $3.5 billion',
 "Asteroid 2025 FA22 Safely Speeds Past Earth at More Than Twice Moon's Distance",
 'Polish missile likely hit house during Russian drone incursion, says minister',
 'Ukraine, US launch fund for critical minerals projects with $150 million investment',
 "Pope Leo plans to keep Francis' key reforms but avoid bigger changes, he says",
 'Europe struggles with chronic drug sh

In [ ]:
# TODO how to get rid of extra symbols in text?

soup_ground_sci = bs4.BeautifulSoup(ground_sci.text, 'lxml')
soup_news_sci = bs4.BeautifulSoup(news_sci.text, 'lxml')

# science news articles
headlines_sci = soup_news_sci.find_all('h4')
headlines_sci = [headline.getText().strip() for headline in headlines_sci]
headlines_sci = headlines_sci[:3]

# ground science news
headlines_ground_sci = soup_ground_sci.find_all('h4')
headlines_ground_sci = [headline.getText() for headline in headlines_ground_sci][1:3]

# merging
headlines_sci.append(headlines_ground_sci)
headlines_sci

['A sixth mass extinction? Not so fast, some scientists say',
 'Horses may have become rideable\xa0with the help of a\xa0genetic mutation',
 'Squashing the spotted lanternfly problem may require enlisting other species',
 ['Dutch Study Challenges Sea Level Rise Projections',
  "Partial Solar Eclipse at the Bottom of the World Closes Out the Year's Sun and Moon Spectacles"]]

#### how to get rid of symbols in text? -> clean text before use

In [46]:
# other websites ....

In [ ]:
### 2 ### LLM integration (perplexity)


